In [43]:
import us

# 9, 8, 7, 2, 3, 5

In [153]:
# 2 Have district results on my own, skipping
# pd.read_csv('data/2_Puhalendran_Seltzer_district_result.docx', encoding='latin-1')

# 3 I have no idea why first name popularity is relevant but ok
# df3 = pd.read_csv('data/3_Sundar_Solai_Susan_Xu_feature.csv', encoding='latin-1')
# df3['state'] = df3['district'].apply(lambda x : x.split(' ')[0].lower().capitalize())
# df3['area'] = 

In [154]:
# 5 Race
df5 = pd.read_csv('data/5_Miruna_Everett_Peter_Minority_Feature.csv', encoding='latin-1').iloc[:,1:]

In [155]:
df5 = (df5.merge(df[['name', 'state_fips', 'district', 'congress_session']], 
                left_on='congressional_district', right_on='name', how='left')
       .drop(['congressional_district', 'candidates','year'], axis=1))

In [156]:
# 7 
df7 = pd.read_csv('data/7_presidential approval ratings  Wanqian, Russell, Eric.csv')
df7['state_fips'] = df7['district'].apply(lambda x : us.states.lookup(x.split('-')[0]).fips)
df7['district'] = df7['district'].apply(lambda x : x.split('-')[1].zfill(2))
df7['lastname'] = df7['name'].apply(lambda x : x.split(', ')[0])
df7.columns = ['raceyear', 'cand_name', 'district', 'pres_approval', 'same_as_pres_party', 'state_fips', 'lastname']
df7['district'] = df7['district'].astype(int)
df7['state_fips'] = df7['state_fips'].astype(int)
df7.loc[df7['raceyear'] == 2016, 'lastname'] = df7.loc[df7['raceyear'] == 2016, 'cand_name'].apply(lambda x : x.split(' ')[-1])

In [157]:
reps = df[['state_fips', 'district', 'raceyear', 'repcandidate']].copy()
reps['lastname'] = reps['repcandidate'].apply(lambda x: x.split(', ')[0] if type(x) is str else None)
reps = reps.merge(df7, on=['raceyear', 'district', 'state_fips', 'lastname'], how='left')

In [158]:
dems = df[['state_fips', 'district', 'raceyear', 'demcandidate']].copy()
dems['lastname'] = dems['demcandidate'].apply(lambda x: x.split(', ')[0] if type(x) is str else None)
dems = dems.merge(df7, on=['raceyear', 'district', 'state_fips', 'lastname'], how='left')

In [159]:
df7 = (reps.drop(['lastname', 'cand_name'], axis=1)
 .merge(dems.drop(['lastname', 'pres_approval', 'cand_name'], axis=1), 
        on=['state_fips', 'district', 'raceyear'], suffixes=('_rep', '_dem'))).sort_values('raceyear')

In [160]:
# 8: weird
df8 = pd.read_csv('data/8_Daniel Qu, Theodore Liu, and Jason Ren.csv')
df8.columns = ['raceyear', 'state', 'district', 'type']
df8['incumbent_party'] = (df8['type'].apply(lambda x : "R" 
                          if x in [0,1,2] else "D" if x in [3,4,5] else "I"))
df8 = df8[['raceyear', 'state', 'district', 'incumbent_party']].copy()

In [161]:
# 9
df9 = pd.read_csv('data/9_Gloria, Nick, Abhi.csv', sep='\t', header=None).iloc[:,:5]
df9.columns = ['raceyear', 'state', 'district', 'candidate', 'med_income']
df9 = df9[['raceyear','state','district','med_income']].drop_duplicates()
df9['log_med_income'] = np.log(df9['med_income'])

In [193]:
df = pd.read_feather('past_house.feather')

/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [194]:
# Only unopposed
df = df[df['repcandidate'].notnull() & df['demcandidate'].notnull()]

In [195]:
def clean_district(x):
    lst = x.split(' ')
    t = lst[1:]
    if len(t) == 1: 
        try:
            return str(int(t[0])).zfill(2)
        except:
            return '00'

In [196]:
df['district'] = df['area'].apply(clean_district)

In [197]:
cols = ['state', 'raceyear', 'district', 'repcandidate', 'repstatus', 
        'demcandidate', 'demstatus', 'pluralityparty', 'repvotesmajorpercent',
        'demvotesmajorpercent']

In [198]:
df = df[cols].copy()

In [199]:
df['rep_score'] = df['repvotesmajorpercent'] / (df['repvotesmajorpercent'] + df['demvotesmajorpercent'])

In [200]:
df['rep_incumbent'] = (df['repstatus'] == 'Incumbent')
df['dem_incumbent'] = (df['demstatus'] == 'Incumbent')

In [201]:
df = df.drop(['repstatus', 'demstatus', 'pluralityparty', 
         'repvotesmajorpercent', 'demvotesmajorpercent'], 
        axis=1)

In [207]:
df.reset_index(drop=True).to_feather('data/main.feather')